# Generate the images in dalle2
 1. Generate an image in dalle2. Inspect the html source and download the webp file without the OpenAI watermark in the dorner. Name the file `1.webp`.
 2. Run the below line to zoom out on the image.
 3. Upload `zoomed.png` to dalle2. "erase" a spot on the outside of the image. The prompt can be modified to change the content of the outfilled portion of the image.
 4. Repeat steps 1-4 until you have the desired results. Name the files 2.webp, 3.webp, etc...
 5. Continue to the next sectin below "Animating the results"

In [ ]:
!convert 1.webp -gravity center -background none -extent 2560x2560 zoomed.png

In [100]:
import numpy as np
import math
from PIL import Image
import ffmpeg

# TODO pull these functions out into helpers.py

def get_percentage_decrease(width1, width2, percent):
    return width2*math.exp(math.log(width1/width2) * (1-percent))

def crop(in_fn, out_fn, new_size):
    im = Image.open(in_fn)
    remove = int(math.ceil((im.width-new_size)/2)) * 2
    size = im.width - remove
    im2 = im.crop(((im.width-size)//2, (im.height-size)//2, (im.width+size)//2, (im.height+size)//2))
    im2.save(out_fn)
        
def resize(in_fn, out_fn, new_size, rs_type=None):
    """rs_type: Image.Resampling.NEAREST is no antialiasing fastest but noisy results,
                Image.Resampling.BOX is good for a little smoothing and only slightly slower,
                Image.Dither.FLOYDSTEINBERG is even more smoothing and a little slower still
    """
    if not rs_type:
        rs_type = Image.Resampling.BOX
    im = Image.open(in_fn)
    im = im.resize((new_size,new_size), rs_type)
    im.save(out_fn)
    
def extend(in_fn, out_fn, new_size):
    im = Image.open(in_fn)
    im = im.convert('RGBA')
    width, height = im.size
    x1 = (new_size - width) // 2
    y1 = (new_size - height) // 2
    new_image = Image.new('RGBA', (new_size, new_size), (0, 0, 0, 0))
    new_image.paste(im, (x1, y1, x1 + width, y1 + height))
    new_image.save(out_fn)
    
# THIS IS BROKEN, USE IM for now
# def overlay(bottom, top, out_fn):
#     im = Image.open(bottom).convert('RGBA')
#     im2 = Image.open(top).convert('RGBA')
#     im.paste(im2, (0, 0), im2)
#     im.save(out_fn)
    
def overlay(bottom, top, out_fn):
    check_output(f'convert {bottom} {top} -gravity center -background None -layers Flatten {out_fn}', shell=True, stderr=STDOUT)

def animate_between(i, delay=10, n=2560, final_size=400, mode='gif', fps=60, filename='final', frames=48):
    # couldn't reproduce this in PIL...
    check_output(f'convert {i-1:02d}.webp -alpha set -virtual-pixel transparent -channel A -blur 0x8 -level 50%,100% +channel soft_edge.png', shell=True, stderr=STDOUT)
    resize(f'{i:02d}.webp', 'resized.png', n)
    extend('soft_edge.png', 'next.png', n)
    overlay('resized.png', 'next.png', 'resized2.png')
    resize('resized2.png', f'cropped_{i}{frames:03d}.png', final_size)
    name = 'resized2.png'
    for j in reversed(range(1, frames)): # start with 0 or 1?
        new_size = int(get_percentage_decrease(n, 1024, (frames-j) / frames))
        print(f'{j}={new_size}', end=', ')
        crop(name, 'cropped.png', new_size)
        resize('cropped.png', f'cropped_{i}{j:03d}.png', final_size)
        if name == 'resized2.png':
            name = 'cropped.png'
    if mode == 'gif':
        check_output(f'convert cropped_*.png -set delay {delay} movie_{i:03d}.gif', shell=True, stderr=STDOUT)
    elif mode == 'mp4':
        # TODO: replace with python ffmpeg code
        check_output(f'ffmpeg -r {fps} -i cropped_{i}%03d.png -y -an movie_{i:03d}.mp4', shell=True, stderr=STDOUT)
    check_output('rm cropped_*.png; rm resized.png; rm soft_edge.png; rm resized2.png; rm next.png; rm cropped.png', shell=True, stderr=STDOUT)
    print('')
    
def generate_full_animation(num_dalle, min_dalle=1, delay=10, n=2560, final_size=400, mode='mp4', fps=60, filename='final', frames=48):
    """Generate a full animation.
    
        num_dalle: the number of dalle outpaint files you created
        min_dalle: the last image that should be processed. If you want to start off at 1.webp, this should be set to 1
        delay: used in gif mode to specify the delay between frames
        n: the outpaint size specified in step 1 above
        final_size: final dimensions of animation
        mode: can be either 'gif' or 'mp4'
        fps: used in mp4 mode to set the speed of the video
        filename: The prefix used for the final output filename
        frames: used in mp4 mode to specify the number of frames generated at each stage of the process
    """
    for i in reversed(range(min_dalle+2, num_dalle+1)):
        print(i, end=': ')
        animate_between(i, delay=delay, n=n, final_size=final_size, mode=mode, fps=fps, frames=frames)
    if mode == 'gif':
        check_output(f'convert movie_*.gif {filename}.gif', shell=True, stderr=STDOUT)
        check_output('rm movie_*.gif', shell=True, stderr=STDOUT)
    elif mode == 'mp4':
        with open('inputs.txt', 'w') as f:
            for i in range(min_dalle+1, num_dalle+1):
                f.write(f'file movie_{i:03d}.mp4\n')
        check_output(f'ffmpeg -y -f concat -i inputs.txt -vcodec copy -acodec copy {filename}.mp4', shell=True, stderr=STDOUT)
        check_output('rm inputs.txt; rm movie_*.mp4', shell=True, stderr=STDOUT)


# Animating the results
 * The `animate_between` function above can be used to generate the frames between each dalle2 output.
 * The `generate_full_animation` function will iterate through all the dalle2 outputs and animate them, and then will combine them into a single animated gif or mp4.
  * If you generate a gif, you can speed up the results by using `gifsicle` to delete frames. See [here](https://graphicdesign.stackexchange.com/a/20937) for an example of how to do this.
  * If you generate an mp4, you can speed up the results by increasing the `fps`.

In [497]:
generate_full_animation(10)

9: 47=2511, 46=2464, 45=2417, 44=2371, 43=2326, 42=2282, 41=2239, 40=2197, 39=2155, 38=2115, 37=2075, 36=2035, 35=1997, 34=1959, 33=1922, 32=1886, 31=1850, 30=1815, 29=1781, 28=1747, 27=1714, 26=1682, 25=1650, 24=1619, 23=1588, 22=1558, 21=1528, 20=1500, 19=1471, 18=1443, 17=1416, 16=1389, 15=1363, 14=1337, 13=1312, 12=1287, 11=1263, 10=1239, 9=1215, 8=1192, 7=1170, 6=1148, 5=1126, 4=1105, 3=1084, 2=1063, 1=1043, 
8: 47=2511, 46=2464, 45=2417, 44=2371, 43=2326, 42=2282, 41=2239, 40=2197, 39=2155, 38=2115, 37=2075, 36=2035, 35=1997, 34=1959, 33=1922, 32=1886, 31=1850, 30=1815, 29=1781, 28=1747, 27=1714, 26=1682, 25=1650, 24=1619, 23=1588, 22=1558, 21=1528, 20=1500, 19=1471, 18=1443, 17=1416, 16=1389, 15=1363, 14=1337, 13=1312, 12=1287, 11=1263, 10=1239, 9=1215, 8=1192, 7=1170, 6=1148, 5=1126, 4=1105, 3=1084, 2=1063, 1=1043, 
